In [ ]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
import cv2
import matplotlib.pyplot as plt
from string import digits
import re
import pandas as pd
import numpy as np

In [ ]:
image = cv2.imread('../test_set/test_set/test_61.jpg')
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
image_labels = pd.read_csv("Label_names.csv")

In [ ]:
def clean_string(string):
    remove_digits = str.maketrans('', '', '\n')
    string = string.translate(remove_digits)
    regex = re.compile('[^a-zA-Z]')
    string = regex.sub('', string)
    return string.lower()
    
clean_string(details)

In [ ]:
custom_config = r'--oem 3 --psm 6'

def get_text(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    details = pytesseract.image_to_string(threshold_img, config=custom_config, lang='eng')
    return clean_string(details)

In [ ]:
image = cv2.imread('../test_set/test_set/test_620.jpg')
get_text(image)

In [ ]:
image_labels.iloc[25]

In [ ]:
label_translate = dict()
for idx in image_labels.index:
    label = image_labels.iloc[idx].values[0].split(' ')
    for l in label:
        if l != "":
            label_translate[l] = idx + 1
label_translate
all_labels = list(label_translate.keys())

In [ ]:
image = cv2.imread('../test_set/test_set/test_620.jpg')
text = get_text(image)
def get_label(text):
    for l in all_labels:
        if l.lower() in text:
            return label_translate[l]
    return 99
            
get_label(text)

In [ ]:
import glob
all_files = glob.glob("../test_set/test_set/test_*")

In [ ]:
c = 0
results = dict()
for path in all_files:
    image = cv2.imread(path)
    text = get_text(image)
    label = get_label(text)
    if label != 99:
        results[path.split('\\')[-1]] = label
    else:
        results[path.split('\\')[-1]] = 1

In [ ]:
output = pd.DataFrame(results.items(), columns=['img_name', 'label'])
print(output)
output.to_csv("sample_submission_text.csv", index=False)